In [ ]:
from pylab import *
import numpy

def make_maze(w, h):
    vis = [[0] * w + [1] for _ in range(h)] + [[1] * (w + 1)]
    ver = [["10"] * w + ['1'] for _ in range(h)] + [[]]
    hor = [["11"] * w + ['1'] for _ in range(h + 1)]
 
    def walk(x, y):
        vis[y][x] = 1
 
        d = [(x - 1, y), (x, y + 1), (x + 1, y), (x, y - 1)]
        shuffle(d)
        for (xx, yy) in d:
            if vis[yy][xx]: continue
            if xx == x: hor[max(y, yy)][x] = "10"
            if yy == y: ver[y][max(x, xx)] = "00"
            walk(xx, yy)
 
    walk(numpy.random.randint(w), numpy.random.randint(h))
 
    s = ""
    for (a, b) in zip(hor, ver):
        s += ''.join(a + ['\n'] + b + ['\n'])
        
    M=[]
    for line in s.split("\n"):
        if line!="":
            R=[]
            for e in line:
                R.append(int(e))
            M.append(R)
    return M

# Configure properties for the maze
maze_width=8
maze_height=8
startLocation=(1,1)
targetLocation=(maze_height*2-1,maze_width*2-1)

# Create the maze
maze=numpy.array(make_maze(maze_width,maze_height))

# Create a function to plot the maze
def plot_maze(maze, start, end, maze_width, maze_height):
    # Plot the maze
    plt.imshow(maze, cmap='binary')
    # Show tick for each square
    plt.xticks(range(0, maze_width*2))
    plt.yticks(range(0, maze_height*2))
    # Show the start and target locations
    plt.plot(start[1], start[0], 'gs')
    plt.plot(end[1], end[0], 'rs')

# Check if the start and target locations are valid
if maze[startLocation[0]][startLocation[1]]==1:
    print('Start location is not valid =', maze[1][1])
else:
    print('Start location is valid =', maze[1][1])
if maze[targetLocation[0]][targetLocation[1]]==1:
    print('Target location is not valid =', maze[15][15])
else:
    print('Target location is valid =', maze[15][15])

# Plot the mission
print(maze)
plt.figure(figsize=[maze_width/2,maze_height/2])
plot_maze(maze, startLocation, targetLocation, maze_width, maze_height)
plt.show()

In [ ]:
def get_neighbors(maze, square):
    x, y = square
    neighbors = []
    if x > 0 and maze[x - 1][y] == 0:
        neighbors.append((x - 1, y))
    if x < len(maze) - 1 and maze[x + 1][y] == 0:
        neighbors.append((x + 1, y))
    if y > 0 and maze[x][y - 1] == 0:
        neighbors.append((x, y - 1))
    if y < len(maze[0]) - 1 and maze[x][y + 1] == 0:
        neighbors.append((x, y + 1))
    return neighbors

In [7]:
# Solve the maze using Q-learning
def solve_maze(maze, start, end, maze_width, maze_height):
    # Initialize the Q-table
    Q = numpy.zeros((maze_height*2, maze_width*2, 4))
    # Set the learning parameters
    lr = .8
    y = .95
    num_episodes = 1000
    # Create lists to contain total rewards and steps per episode
    rList = []
    for i in range(num_episodes):
        # Reset environment and get first new observation
        s = start
        rAll = 0
        d = False
        j = 0
        # The Q-Table learning algorithm
        while j < 99:
            j += 1
            # Choose an action by greedily (with noise) picking from Q table
            a = numpy.argmax(Q[s[0]][s[1]] + numpy.random.randn(1, 4) * (1. / (i + 1)))
            # Get new state and reward from environment
            s1 = get_neighbors(maze, s)[a]
            r = -1
            # Update Q-Table with new knowledge
            Q[s[0]][s[1]][a] = Q[s[0]][s[1]][a] + lr * (r + y * numpy.max(Q[s1[0]][s1[1]]) - Q[s[0]][s[1]][a])
            rAll += r
            s = s1
            if s == end:
                break
        rList.append(rAll)
    # Print the Q-table
    print("Final Q-Table Values")
    print(Q)
    # Plot the rewards per episode
    plt.plot(rList)
    plt.show()
    # Print the total number of steps
    print("Total Steps: " + str(j))
    # Return the Q-table
    return Q